# Markov chain text generator

This Markov chain model takes input in the form of .txt files, and uses prompts to generate new text.

This Jupyter notebook goes step by step through the process of creating a text generating application.
This notebook is intended for beginners without a strong coding background.
As such, most of the explanation has to do with basic aspects of the code.
The notebook doesn't go into much detail about all the code, especially the more complex functions that make up the text generator.

The code was adapted from from Luciano (StrikingLoo's) [ASOIAF-Markov repository](https://github.com/StrikingLoo/ASOIAF-Markov).
He also wrote an article called ['Markov Chains: How to Train Text Generation to Write Like George R. R. Martin'](https://www.kdnuggets.com/2019/11/markov-chains-train-text-generation.html), where he goes into more detail around what Markov chains are and how they work in the context of his text generator.

Another great resource for learning about different methods of machine learning (including Markov chains) is the book [You look like a thing and I love you](https://www.janelleshane.com/book-you-look-like-a-thing) by Janelle Shane.
The book is a good introduction to Artificial Intelligence full of humour and meant for a broad audience (without assumptions about previous coding skills).

## Step 1 : get libraries ready

The following code may be required if any of the libraries that we need to import are not installed.
'Libraries' refer to code that someone else has written and made available for reuse.
We can simply add that code into our project by 'importing' the library, rather than having to write all that code from scratch.
By default, in this notebook the code has been commented out (the # symbol before the code means that the computer will skip that line of code).
In order to run the code, if needed, we need to remove the # symbol and run the cell.

In [ ]:
# !pip install pandas
# !pip install seaborn
# !pip install numpy
# !pip install glob
# !pip install scipy

The following code will import the libraries that we will use to run this program.

When you run the code below, you may get an error if you are missing a specific library.
If that is the case, you can install the missing library by running the code in the cell above.
Once you have installed the missing library (or libraries), you can run the cell below again.

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import glob
import scipy

## Step 2 : get corpus data ready

The following code indicates the path to the corpus files.
It assumes that the sub-directory (folder) called 'documents' is in the same directory as the Jupyter notebook.

The code also indicates that, within the 'documents' directory, we are interested only in text files (anything that has a .txt extension).

In [ ]:
file_names = glob.glob('documents/*.txt')

# If your documents are stored in a different directory on your computer, you will need to write the full path to retrieve the files as in the following example:
#
# file_names = glob.glob('C:/Users/example/Desktop/documents/*.txt')
#
# If you need to use this option, you must also comment-out the first line of code in this block, and un-comment the line that has the full path.

The following code will tell us how many text files (files with a .txt extension) are in the corpus directory.

In [ ]:
len(file_names)

## Step 3 : parse the text from the corpus files

The following code takes the text in our corpus files and parses it out into sentences, using the period as the delimiter criteria.
It does additional cleanup of the text by removing line breaks and tabs from the text.

In [ ]:
def get_sentences(file_name):
    with open(file_name, 'r', encoding='utf-8') as f:
        return f.read().split('.')

In [ ]:
MIN_LENGTH = 15
sentences = []
for file_name in file_names:
    sentences+=get_sentences(file_name)

sentences = [sentence.replace('\n','') for sentence in sentences]
sentences = [sentence.replace('\t','') for sentence in sentences]
sentences = [sentence for sentence in sentences if len(sentence)>MIN_LENGTH]

In [ ]:
lengths = [len(sentence) for sentence in sentences]

In [ ]:
lengths = pd.Series(lengths)

In [ ]:
lengths.quantile(.8)

In [ ]:
lengths.describe()
# 14228 our of 18k

The following code returns the parsed-out sentences.

In [ ]:
sentences

## Step 4 : load whole corpus

The following code iterates through all the files in the corpus and performs basic cleanup tasks.
A few lines of code return basic data about our corpus, such as total number of words, and total number of unique words.

In [ ]:
corpus = ""
for file_name in file_names:
    with open(file_name, 'r', encoding='utf-8') as f:
            corpus+=f.read()
corpus = corpus.replace('\n',' ')
corpus = corpus.replace('\t',' ')
corpus = corpus.replace('“', ' " ')
corpus = corpus.replace('”', ' " ')
for spaced in ['.','-',',','!','?','(','—',')']:
    corpus = corpus.replace(spaced, ' {0} '.format(spaced))
len(corpus)

In [ ]:
len(corpus)

The following code outputs a sample of words from the corpus that we built.

In [ ]:
corpus[1000:1500]

The following code splits the corpus into individual words, then counts the number of words.

In [ ]:
corpus_words = corpus.split(' ')
corpus_words= [word for word in corpus_words if word != '']

In [ ]:
len(corpus_words)

The following code outputs all the words that are part of the corpus.

In [ ]:
corpus_words

In [ ]:
len(corpus_words)

The following code indicates the total number of unique words in the corpus.

In [ ]:
distinct_words = list(set(corpus_words))
word_idx_dict = {word: i for i, word in enumerate(distinct_words)}
distinct_words_count = len(list(set(corpus_words)))
distinct_words_count

## Step 5 : build matrix for Markov chain

This code will create the matrix that serves as the basis for generating new text, by calculating the most likely co-occurrences of words based on the corpus.

In [ ]:
next_word_matrix = np.zeros([distinct_words_count,distinct_words_count])

In [ ]:
word_idx_dict

In [ ]:
for i, word in enumerate(corpus_words[:-1]):
    first_word_idx = word_idx_dict[word]
    next_word_idx = word_idx_dict[corpus_words[i+1]]
    next_word_matrix[first_word_idx][next_word_idx] +=1

In [ ]:
def most_likely_word_after(aWord):
    most_likely = next_word_matrix[word_idx_dict[aWord]].argmax()
    return distinct_words[most_likely]

## Step 6 : test the code to obtain sample sentences

The following code prepares the program to print some sample sentences:

By default, the sample sentences will be 15 words long (the value of 'length').
You may replace the length value to a different integer in order to get longer or shorter output sentences.

Default length = 15:

`def naive_chain(seed, length=15)`

Modified example, length = 8:

`def naive_chain(seed, length=8)`

In [ ]:
def naive_chain(seed, length=15):
    current_word = seed
    sentence = seed

    for _ in range(length):
        sentence+=' '
        next_word = most_likely_word_after(current_word)
        sentence+=next_word
        current_word = next_word
    return sentence

The following code will print sample sentences.

Given an initial prompt word (seed), the program calculates the most likely word to occur after, and iterates through this process until it reaches the length of the sentence that was defined above.

In the code below, you can alter the seed value(s) to see what happens if you use other words as the prompt.

Example:

Default = `print(naive_chain('the'))`

Modified = `print(naive_chain('when'))`

The code may crash if the prompt that you use does not match any of the words that form part of the original corpus.

In [ ]:
print(naive_chain('the'))
print(naive_chain('I'))
print(naive_chain('he'))
print(naive_chain('she'))

In [ ]:
import random
from random import random 

def weighted_choice(objects, weights):
    """ returns randomly an element from the sequence of 'objects', 
        the likelihood of the objects is weighted according 
        to the sequence of 'weights', i.e. percentages."""

    weights = np.array(weights, dtype=np.float64)
    sum_of_weights = weights.sum()
    # standardization:
    np.multiply(weights, 1 / sum_of_weights, weights)
    weights = weights.cumsum()
    x = random()
    for i in range(len(weights)):
        if x < weights[i]:
            return objects[i]

In [ ]:
from numpy.random import choice

def sample_next_word_after(aWord, alpha = 0):
    next_word_vector = next_word_matrix[word_idx_dict[aWord]] + alpha
    likelihoods = next_word_vector/next_word_vector.sum()
    return weighted_choice(distinct_words, likelihoods)

In [ ]:
sample_next_word_after('the')

In [ ]:
def stochastic_chain(seed, length=15):
    current_word = seed
    sentence = seed

    for _ in range(length):
        sentence+=' '
        next_word = sample_next_word_after(current_word)
        sentence+=next_word
        current_word = next_word
    return sentence

In [ ]:
stochastic_chain('table')

In [ ]:
'John W . I had feasted them mutton , " You did not even the Eyrie'
'the Seven in front of whitefish in a huge blazes burning flesh . I had been'
'a squire , slain , they thought . " He bathed in his head . The'
'Bran said Melisandre had been in fear I’ve done . " It must needs you will'
'Melisandre would have feared he’d squired for something else I put his place of Ser Meryn'
'Daenerys is dead cat - TOOTH , AT THE GREAT , Asha , which fills our'
'Daenerys Targaryen after Melara had worn rich grey sheep to encircle Stannis . " The deep'


In [ ]:
k = 5
sets_of_k_words = [ ' '.join(corpus_words[i:i+k]) for i, _ in enumerate(corpus_words[:-k]) ]

print([len(list(set(sets_of_k_words))),
       len(sets_of_k_words)])

In [ ]:
from scipy.sparse import dok_matrix

sets_count = len(list(set(sets_of_k_words)))
next_after_k_words_matrix = dok_matrix((sets_count, len(distinct_words)))
print(next_after_k_words_matrix.shape)

In [ ]:
distinct_sets_of_k_words = list(set(sets_of_k_words))
k_words_idx_dict = {word: i for i, word in enumerate(distinct_sets_of_k_words)}
distinct_k_words_count = len(list(set(sets_of_k_words)))
print(len(sets_of_k_words))
for i, word in enumerate(sets_of_k_words[:-k]):
    if i % 50000 == 0:
        print(i)
    word_sequence_idx = k_words_idx_dict[word]
    next_word_idx = word_idx_dict[corpus_words[i+k]]
    next_after_k_words_matrix[word_sequence_idx, next_word_idx] +=1

In [ ]:
def stochastic_chain(seed, chain_length=15, seed_length=2):
    current_words = seed.split(' ')
    if len(current_words) != seed_length:
        raise ValueError(f'wrong number of words, expected {seed_length}')
    sentence = seed

    for _ in range(chain_length):
        sentence+=' '
        next_word = sample_next_word_after_sequence(' '.join(current_words))
        sentence+=next_word
        current_words = current_words[1:]+[next_word]
    return sentence

In [ ]:
from numpy.random import choice

def sample_next_word_after_sequence(word_sequence, alpha = 0):
    next_word_vector = next_after_k_words_matrix[k_words_idx_dict[word_sequence]] + alpha
    likelihoods = next_word_vector/next_word_vector.sum()
    return weighted_choice(distinct_words, likelihoods.toarray())

In [ ]:
stochastic_chain('the world')

In [ ]:
stochastic_chain('Jon Snow')

In [ ]:
stochastic_chain('Eddard Stark')

In [ ]:
stochastic_chain('The game')

In [ ]:
stochastic_chain('The game')

In [ ]:
stochastic_chain('I have')

In [ ]:
stochastic_chain('heard the')

In [ ]:
stochastic_chain('that made them look like', 15, 5)

In [ ]:
distinct_sets_of_k_words[:10]

In [ ]:
'Maybe we should all do the same , Jon reflected glumly . He made himself eat , hungry or no'